## Attributes Classificator

In this notebook I will try to classify all the attributes present in a face.
This could be useful to use the attributes as condition in input of conditional models.

A first attempt was made trying to train from scratch a VGGNetLike model. Unfortunately the accuracy remained very low. I think I will try to switch to transfer learning on a pretrained model. 

In [15]:
from classes.Architectures import VGGNetLike
import numpy as np
import pandas as pd
from os.path import join as opj

import os

import tensorflow as tf
from tensorflow import keras
import numpy as np
import wandb
from wandb.keras import WandbCallback
from imutils import paths
from tensorflow.data import AUTOTUNE
import matplotlib.pyplot as plt

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [2]:
images_dir=r"/home/matteo/NeuroGEN/Dataset/Img/img_align_celeba"
anno_dir=r"/home/matteo/NeuroGEN/Dataset/Anno/list_attr_celeba.csv"
#other important definitions

EPOCHS=50
BS=64
INIT_LR=1e-4

config={}
config["epochs"]=EPOCHS
config["BS"]=BS
config["init_lr"]=INIT_LR

wandb.init(project="TorVergataExperiment-Generative",config=config,name="CelebA_Attribute_Prediciton")

## Load data and indices

In [3]:
df=pd.read_csv(anno_dir,sep=",")

In [4]:
@tf.function
def load_images(imagePath,attr):

    #attr=df.iloc[idx].drop("image_id",axis=1).values
    # read the image from disk, decode it, resize it, and scale the
    # pixels intensities to the range [0, 1]
    image = tf.io.read_file(imagePath)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224, 224)) / 255.0

    #eventually load other information like attributes here
    
    # return the image and the extra info
    
    
    return image,attr

In [34]:
print("[INFO] loading image paths...")
imagePaths = list(paths.list_images(images_dir))


train_len=int(0.8*len(imagePaths))
val_len=int(0.1*len(imagePaths))
test_len=int(0.1*len(imagePaths))

train_imgs=imagePaths[:train_len]                                #      80% for training
val_imgs=imagePaths[train_len:train_len+val_len]                 #      10% for validation
test_imgs=imagePaths[train_len+val_len:]                         #      10% for testing

train_idxs=[int(i.split("/")[-1].split(".jpg")[0])-1 for i in train_imgs]
val_idxs=[int(i.split("/")[-1].split(".jpg")[0])-1 for i in val_imgs]
test_idxs=[int(i.split("/")[-1].split(".jpg")[0])-1 for i in test_imgs]

train_attr=[df.iloc[idx].drop("image_id").values for idx in train_idxs]
val_attr=[df.iloc[idx].drop("image_id").values for idx in val_idxs]
test_attr=[df.iloc[idx].drop("image_id").values for idx in test_idxs]


print(f"[TRAINING]\t {len(train_imgs)}\n[VALIDATION]\t {len(val_imgs)}\n[TEST]\t\t {len(test_imgs)}")

[INFO] loading image paths...
[TRAINING]	 138545
[VALIDATION]	 17318
[TEST]		 17319


In [35]:
train_attr=np.array(train_attr).astype("float")
val_attr=np.array(val_attr).astype("float")
test_attr=np.array(test_attr).astype("float")

In [36]:
## sigmoid adaptation

train_attr[train_attr==-1]=1.
val_attr[val_attr==-1.]=1.
test_attr[test_attr==-1.]=1.

In [31]:
train_attr.shape,len(train_imgs)

((138545, 40), 138545)

In [37]:
#TRAINING 

train_dataset = tf.data.Dataset.from_tensor_slices((train_imgs,train_attr))
train_dataset = (train_dataset
    .shuffle(1024)
    .map(load_images)
    .cache()
    .repeat()
    .batch(BS)
    .prefetch(AUTOTUNE)
)

ts=len(train_imgs)//BS

##VALIDATION

val_dataset = tf.data.Dataset.from_tensor_slices((val_imgs,val_attr))
val_dataset = (val_dataset
    .shuffle(1024)
    .map(load_images)
    .cache()
    .repeat()
    .batch(BS)
    .prefetch(AUTOTUNE)
)

vs=len(val_imgs)//BS

## TEST

test_dataset = tf.data.Dataset.from_tensor_slices((test_imgs,test_attr))
test_dataset = (test_dataset
    .shuffle(1024)
    .map(lambda x,y:load_images(x,y), num_parallel_calls=AUTOTUNE)
    .cache()
    .batch(BS)
    .prefetch(AUTOTUNE)
)

In [ ]:
for x,y in train_dataset:
    print(x.shape)
    print(y)
    break

## Define the model

In [9]:
base= MobileNetV2(input_shape=(224,224,3),
    alpha=1.0,
    include_top=False,
    weights="imagenet",
    pooling='avg'
)

# model top
x = base.output
x = Dense(1000, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
top =Dense(40, activation='sigmoid')(x)

model=Model(base.input,top)
model.summary()

9412608/9406464 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
_______________________________

In [39]:
#model=VGGNetLike((128,128,3),40)
#opt=tf.keras.optimizers.Adam(INIT_LR)
#model.compile(loss="categorical_crossentropy",optimizer=opt,metrics="accuracy")
@tf.function
def cosine_proximity(y_true, y_pred):
    y_true = K.l2_normalize(y_true, axis=-1)
    y_pred = K.l2_normalize(y_pred, axis=-1)
    return -K.sum(y_true * y_pred, axis=-1)


model.compile(loss=cosine_proximity,
              optimizer='adadelta',
              metrics='binary_accuracy')

##callbacks


es=tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=5,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)

check=tf.keras.callbacks.ModelCheckpoint(
    "models/attributeclassifier",
    monitor="val_loss",
    verbose=0,
    save_best_only=False,
    save_weights_only=False,
    mode="auto",
    save_freq="epoch",
    options=None,
)

callbacks=[WandbCallback(),es,check]


## Train the model

In [ ]:
model.fit(train_dataset,epochs=EPOCHS,steps_per_epoch=ts,callbacks=callbacks,validation_data=val_dataset)

Epoch 1/50
  73/2164 [>.............................] - ETA: 29:56 - loss: -0.8723 - binary_accuracy: 0.4997